In [1]:
!nvidia-smi

Sun Aug 11 07:33:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

  Using cached accelerate-0.33.0-py3-none-any.whl.metadata (18 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
Using cached accelerate-0.33.0-py3-none-any.whl (315 kB)
Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1
Found existing installation: transformers 4.44.0
Uninstalling transformers-4.44.0:
  Successfully uninstalled transformers-4.44.0
Found existing installation: accelerate 0.33.0
Uninstalling accelerate-0.33.0:
  Successfully uninstalled accelerate-0.33.0
  Using cached transformers-4.44.0-py3-none-any.whl.metadata (43 kB)
  Using cached accelerate-

In [4]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk, load_metric
import matplotlib.pyplot as plt, pandas as pd

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk  # NLP processing library
from nltk.tokenize import sent_tokenize

from tqdm import tqdm  #progress bar
import torch

nltk.download("punkt") # a pre-trained Punkt tokenizer for English

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
model_ckpt = "Falconsai/medical_summarization"  # T5 Large for Medical Text Summarization https://huggingface.co/Falconsai/medical_summarization
# model_ckpt = "google/pegasus-cnn_dailymail" # model checkpoint from Hugging Face

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_T5 = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [7]:
from datasets import load_dataset
##  https://www.kaggle.com/datasets/mathurinache/ms2multidocuments (13.9 GB size. Too large dataset size so not used in this project)
## https://huggingface.co/datasets/owkin/medical_knowledge_from_extracts (only 4.16 MB size)
dataset = load_dataset("owkin/medical_knowledge_from_extracts")
dataset

Generating train split:   0%|          | 0/1383 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 1383
    })
})

In [8]:
##### Splitting dataset into train, test and validation

from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# Convert the dataset to a pandas DataFrame
df = dataset['train'].to_pandas()

# def split_data(self,dataset):
# Convert the dataset to a pandas DataFrame
df = dataset['train'].to_pandas()

# Function to extract text between 'Input:' and 'Output:'
def extract_text_between(input_string):
    try:
        # Find the start of 'Input:' and end of 'Output:'
        start = input_string.index('Input:') + len('Input:')
        end = input_string.index('Output:')
        return input_string[start:end].strip()  # Extract and trim whitespace
    except ValueError:
        return None  # Return None if 'Input:' or 'Output:' is not found

# Apply the function to the DataFrame column
df['Question'] = df['Question'].apply(extract_text_between)

# Split the data into train and temp sets (80% train, 20% temp)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)

# Further split the temp set into validation and test sets (50% of temp each, i.e., 10% of the original data each)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Convert DataFrames to Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Combine into a DatasetDict
split_dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

print(split_dataset)
dataset = split_dataset

DatasetDict({
    train: Dataset({
        features: ['Question', 'Answer', '__index_level_0__'],
        num_rows: 1106
    })
    validation: Dataset({
        features: ['Question', 'Answer', '__index_level_0__'],
        num_rows: 138
    })
    test: Dataset({
        features: ['Question', 'Answer', '__index_level_0__'],
        num_rows: 139
    })
})


In [13]:
print(len(dataset['train']['Question'][1]))
dataset['train']['Question'][1]

2005


'Immunogenicity and safety of 10-valent pneumococcal non-typeable Haemophilus influenzae protein D-conjugate vaccine (PHiD-CV) co-administered with routine childhood vaccines in Taiwan.  The immunogenicity and safety of the 10-valent pneumococcal nontypeable Haemophilus influenzae (H. Influenzae) protein D conjugate vaccine (PHiD-CV), co-administered with routine childhood vaccines, were assessed in Taiwanese infants. In this open study, 230 healthy infants were primed with three doses of PHiD-CV (Synflorix) and diphtheria, tetanus, acellular pertussis, hepatitis B (HBV), inactivated poliomyelitis and Haemophilus influenzae type b (Hib) conjugate vaccine (DTPa-HBV-IPV/Hib vaccine) at 1.5, 3 and 6 months of age and two doses of oral human rotavirus vaccine at 1.5 and 3 months. Pneumococcal immune responses were assessed 1 month post-dose three, by 22F-inhibition ELISA and opsonophagocytic activity (OPA) assay. Local and general solicited/unsolicited symptoms and serious adverse events (

In [14]:
dataset['train']['Answer'][1]

"{'conditions': 'Pneumococcal Disease|Streptococcus Pneumoniae Vaccines', 'interventions': 'Biological: Synflorix|Biological: Infanrix hexa|Biological: Rotarix'}"

In [9]:
split_lengths = [len(dataset[split])for split in dataset]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset['train'].column_names}")
print("\nDialogue:")

print(dataset["test"][1]["Question"])

print("\nSummary:")

print(dataset["test"][1]["Answer"])

Split lengths: [1106, 138, 139]
Features: ['Question', 'Answer', '__index_level_0__']

Dialogue:
Particle size matters: diagnostics and treatment of small airways involvement in asthma.  Small airways are an important site of inflammation and obstruction in asthma, which contributes to the severity of airway hyperresponsiveness (AHR) that is usually measured by nebulisation of large-particle stimuli. We investigated whether small and large particle sizes of aerosolised adenosine monophospate (AMP) provoke similar severity of AHR. Additionally, effects of the small-particle inhaled corticosteroid (ICS) ciclesonide and large-particle ICS fluticasone on AHR to large- and small-particle size AMP were assessed. After a 4-week run-in period using open-label fluticasone (100 μg b.i.d.), 37 mild-to-moderate asthmatics underwent provocations with standard-size (3.7 μm), large-particle (9.9 μm) and small-particle (1.06 μm) AMP. Subjects received 4-week ciclesonide (160 μg s.i.d.) or fluticasone 

In [10]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['Question'], truncation = False, padding=True)

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['Answer'], max_length = 256, truncation = False,padding=True)

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }


In [11]:
dataset_pt = dataset.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/1106 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1334 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Map:   0%|          | 0/138 [00:00<?, ? examples/s]

Map:   0%|          | 0/139 [00:00<?, ? examples/s]

In [12]:
dataset_pt["train"]

Dataset({
    features: ['Question', 'Answer', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1106
})

In [13]:
# Training

from transformers import DataCollatorForSeq2Seq
# Data collator that will dynamically pad the inputs received, as well as the labels.
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_T5)

In [14]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='T5_output', num_train_epochs=40, #learning_rate = 1e-4,
    warmup_steps=500,# Number of steps used for a linear warmup from 0 to learning_rate. Overrides any effect of warmup_ratio.
    per_device_train_batch_size=16, per_device_eval_batch_size=16,
    weight_decay=0.01, #  The weight decay to apply (if not zero) to all layers except all bias and LayerNorm weights in AdamW optimizer.
    logging_steps=10, # Number of update steps between two logs if logging_strategy="steps".
    evaluation_strategy="steps", # Evaluation strategy to adopt during training ('no', 'steps', 'epoch').
    eval_steps=500, #  Number of update steps between two evaluations if evaluation_strategy="steps". Defaults to the same value as logging_steps if not set.
    save_steps=1e6, # Number of updates steps before two checkpoint saves if save_strategy="steps".
    gradient_accumulation_steps=16, # Number of updates steps to accumulate the gradients for, before performing a backward/update pass.
    # fp16 = True,  # Whether to use fp16 16-bit (mixed) precision training instead of 32-bit training.
    # group_by_length = True, # group together samples of roughly the same length in the training dataset (to minimize padding applied and be more efficient)
)
'''
output_dir: Directory where the model checkpoints and logs will be saved.
num_train_epochs: Number of epochs to train the model.
warmup_steps: Number of steps to perform learning rate warmup.
per_device_train_batch_size: Batch size for training per device.
per_device_eval_batch_size: Batch size for evaluation per device.
weight_decay: Weight decay for regularization.
logging_steps: Number of steps between logging updates.
evaluation_strategy: Evaluation strategy during training ('steps' means evaluation will be done every eval_steps steps).
eval_steps: Number of steps between evaluations.
save_steps: Number of steps between saving model checkpoints. 1e6 is a very large number, effectively disabling checkpoint saving.
gradient_accumulation_steps: Number of steps to accumulate gradients before performing a backward pass.
'''

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


"\noutput_dir: Directory where the model checkpoints and logs will be saved.\nnum_train_epochs: Number of epochs to train the model.\nwarmup_steps: Number of steps to perform learning rate warmup.\nper_device_train_batch_size: Batch size for training per device.\nper_device_eval_batch_size: Batch size for evaluation per device.\nweight_decay: Weight decay for regularization.\nlogging_steps: Number of steps between logging updates.\nevaluation_strategy: Evaluation strategy during training ('steps' means evaluation will be done every eval_steps steps).\neval_steps: Number of steps between evaluations.\nsave_steps: Number of steps between saving model checkpoints. 1e6 is a very large number, effectively disabling checkpoint saving.\ngradient_accumulation_steps: Number of steps to accumulate gradients before performing a backward pass.\n"

In [15]:

trainer = Trainer(model=model_T5, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_pt["train"],
                  eval_dataset=dataset_pt["validation"])

In [16]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 870.00 MiB. GPU 

In [ ]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

C:\Users\Admin\AppData\Local\Temp\ipykernel_6376\2696170338.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
c:\Users\Admin\Documents\Manu\Jupyter_notebooks\NLP\text_summarization\venv\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
score = calculate_metric_on_test_ds(
    dataset['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'Question', column_summary= 'Answer'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'Falconsai_T5'] )

100%|██████████| 5/5 [06:43<00:00, 80.78s/it]


,rouge1,rouge2,rougeL,rougeLsum
Falconsai_T5,0.002079,0.0,0.002079,0.002079


In [ ]:
## Save model
model_T5.save_pretrained(model_ckpt)

In [ ]:
## Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [ ]:
#Load

tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [ ]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = dataset["test"][0]["Question"]

reference = dataset["test"][0]["Answer"]

pipe = pipeline("summarization", model=model_ckpt,tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Token indices sequence length is longer than the specified maximum sequence length for this model (680 > 512). Running this sequence through the model will result in indexing errors


Dialogue:
Your goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.

```TypeScript

 { // Information on the clinical trial characteristics from the abstract
 conditions: string // The condition or disease being treated in the clinical trial
 drug_or_intervention: string // The drug or intervention used in the clinical trial
}
```


Please output the extracted information in JSON format. Do not output anything except for the extracted information. Do not add any clarifying information. Do not add any fields that are not in the schema. If the text contains attributes that do not appear in the schema, please ignore them. All output must be in JSON format and follow the schema specified above. Wrap the JSON in <json> tags.



Input:Responses to adalimumab in patients with active psori